In [ ]:
# number of partipants
'''

given through number of files in calendar
Also in the description
48 participants

'''

# Days recorded per participant

'''

estimated: 3/24/2013
through
estimated: 6/14/2013


'''

# Demographic diversity (age, gender)

'''
48 undergrads and grad students
doesn't specify
'''

# Total datapoints and labels 

'''

need code to go through each folder and file

'''

# Datapoints per label

'''

need code to go through each folder and file

'''

# Input streams (accelerometer, gyro, annotations, etc.) 

'''
activity
audio
conversation
bluetooth
gps
phonecharge
phonelock
wifi
wifi_location

'''